# TP Les pays du monde - Recherche dans une table

### Previously on NSI

Dans le TP précédent, nous avons appris à charger les données d'une table en mémoire vive dans notre programme.  

Vous trouverez un exemple de correction dans le fichier `chargement.py`. Ce fichier contient une fonction `chargePays` qui permet de charger la table des pays en donnant une `list` de `namedtuple`.

**Attention:** la fonction `chargePays` ne permet d'importer que les données de `pays.csv`. Elle échouera à charger une table qui n'a pas exactement le mêmes nombres de colonnes ou dont les colonnes ne seront pas exactement du même type (entier, nombres à virgules). Une fonction pour charger d'autres tables sera fournie ultérieurement.

Ce fichier contient aussi, dans une fonction `main` un petit programme qui affiche la table complète ainsi que la table restreinte aux seuls pays d'Europe (dernière étape du TP précédent). Cette partie du programme utilise le module `affichage.py` aussi fourni.

Le module `affichage.py` contient une fonction `afficher_table` qui prend en argument:
- la table à afficher (une `list` de `namedtuple`, `tuple` ou `list`)
- le titre sous forme d'un `str` du titre à afficher avant la table (à la fois en haut de la fenêtre et dans la barre de titre). Cet argument est optionnel.
- l'indice (`int`) de la première ligne que l'on veut afficher. Par défaut: 0
- l'indice (`int`) de la dernière ligne que l'on veut afficher. Si fin est à None (valeur par défaut), on affiche jusqu'à la dernière ligne.

Cette fonction affiche la table dans une fenêtre `tkinter`. Vous pouvez utiliser les deux modules `affichage.py` et `chargement.py` dans ce TP. Vous pouvez aussi utiliser le TP précédent pour le chargement de la table et/ou écrire votre propre programme pour afficher une table.

Le cellule suivante vous permet de copier-coller les fonctions utiles du TP précédent. Vous pouvez aussi les copier dans un fichier `.py` et vous contenter de l'importer. Dans ce cas, ce fichier devra être dans le même dossier que ce notebook.

In [2]:
from collections import namedtuple
def chargePays(cheminFichier):
    """
    Fonction pour charger la table "pays.csv".

    Argument:
            - cheminFichier: str du chemin du fichier à charger

    Renvoi:
            Les données chargées sous forme d'une list de namedtuple.

    Le choix du namedtuple permet un accès aux données d'une ligne
    de la table à partir du numéro de colonne ou du nom du champ
    correspondant à cette colonne.

    Le fichier doit être cohérent avec la table attendue
    """
    # Ouverture du fichier en lecture:
    # L'argument encoding permet d'utiliser le bon charset lors du chargement du fichier. Les versions récentes de python devraient utiliser UTF-8 par défaut, mais cette précaution assure le compatibilité lors de l'ouverture.
    with open('pays.csv', "r", encoding="UTF-8") as fichier:
        # Création du namedtuple Pays:
        Pays = namedtuple("Pays", ["Code", "Nom", "Continent", "Region",  "Aire", "AnneeIndependance", "Population",
                          "EsperanceVie", "PIB", "PIBAncien", "NomLocal", "FormeGouvernement", "ChefEtat", "Capitale", "Code2"])
        # List pour accueillir les données de la table
        table = []

        for ligne in fichier:
            # Il faut penser à enlever le retour à la ligne en fin de ligne
            ligne = ligne.rstrip("\n")
            # On crée la liste des éléments en coupant au séparateur ";"
            cellules = ligne.split(";")
            # Il faut convertir les cellules dans le type voulu
            # Pour le faire de manière automatique, on utilise un tuple des types attendus.
            types = (str, str, str, str, float, int, int, float,
                     float, float, str, str, str, int, str)
            for i in range(len(cellules)):
                if cellules[i] == "NULL":  # Précaution pour gérer les NULL
                    cellules[i] = None
                else:
                    # Equivalent à str(cellules[i]), int(cellules[i]), float(cellules[i]),... selon le type rencontré dans types
                    cellules[i] = types[i](cellules[i])
            # Ajout d'un nouveau namedtuple dans la table:
            # Le * unpack la list en ses composants.
            table.append(Pays(*cellules))

    return table

from affichage import * 

## Recherches dans la table

Maintenant que vous savez charger la table et que vous avez déjà fait une recherche à l'intérieur, nous allons nous servir de la table pour y effectuer des recherches et en extraire de l'information.

Si vous utilisez `chargement.py`, la table est une `list` de `namedtuple` dont les champs sont, dans cet ordre:
`"Code", "Nom", "Continent", "Region",  "Aire", "AnneeIndependance", "Population", "EsperanceVie", "PIB", "PIBAncien", "NomLocal", "FormeGouvernement", "ChefEtat", "Capitale", "Code2"`

### Les monarchies

Afficher la table de tous les pays dont la forme de gouvernement est une monarchie (`"Monarchy"`)

In [10]:
pays = chargePays("pays.csv")
monarchy = [(p.Nom, p.FormeGouvernement) for p in pays if "Monarchy" in p.FormeGouvernement]
afficher_table(monarchy, "Les pays d'une monarchie")

### Petit pays

Afficher la table de tous les pays dont la surface fait moins de 100 km².

In [31]:
petitpays = [(p.Nom, p.Aire) for p in pays if p.Aire < 100]
afficher_table(petitpays, "Les plus petit pays du Monde")

### Là où on vit vieux

Donner table de tous les pays dans lesquels l'espérance de vie est supérieure à 79 ans.

**Attention:** L'espérance de vie peut avoir la valeur `None` pour certains pays. Or, on ne peut pas comparer `None` à un nombre.

In [32]:
lesvieux = [(p.Nom, p.EsperanceVie) for p in pays if p.EsperanceVie and p.EsperanceVie >= 79]
afficher_table(lesvieux, "Les pays dans lesquels on vit VIEUX")

### Là où on a moins de chance

Donner **le** pays dans lesquels l'espérance de vie est la plus faible.

In [35]:
def pays_esperance_vie_faible(pays):
    """
    Renvoie le namedtuple du pays avec la plus faible espérance de vie.

    Arguments:
    - pays : list de namedtuple de pays

    Renvoi:
    - namedtuple : le namedtuple du pays avec la plus faible espérance de vie

    """
    # Initialisation du pays avec la plus faible espérance de vie
    pays_faible_esperance_vie = pays[0]

    # Parcours de tous les pays pour trouver celui avec la plus faible espérance de vie
    for p in pays:
        if p.EsperanceVie is not None and p.EsperanceVie < pays_faible_esperance_vie.EsperanceVie:
            pays_faible_esperance_vie = p

    return pays_faible_esperance_vie


afficher_table([pays_esperance_vie_faible(pays)], "Pays dont l'espérance de vie est la plus faible")


In [36]:
#Attention en cas d'ex-aequo ne pas test l'égalité 

a = 0 
for i in range(10):
    a += 0.1
print(a)

#a == 1.0 FAUX
#abs(a-1.0) < 0.0000001 VRAI

0.9999999999999999


### Son boulot de dans plein de pays

Donner la table de tous les pays dont le chef d'état est le président français.

In [22]:
president_francais = [(p.Nom, p.ChefEtat) for p in pays if p.ChefEtat == "Jacques Chirac"]
afficher_table(president_francais, "Les pays où le président est le président français")

### Le même en bref

La table précédente contenant beaucoup de colonnes, n'affichez que les colonnes du nom du pays, du continent et du chef d'état (pour vérifier que c'est toujours le même).

In [23]:
def construire_table_pays_continent_chef(pays):
    """
    Construit une table de pays avec leur continent et chef d'état.
    Arguments :
        pays : une liste de tuples contenant le nom, le continent et le chef d'état de chaque pays.
    Returns :
        Une liste de namedtuple PaysAffichage contenant le nom, le continent et le chef d'état de chaque pays.
    """
    PaysAffichage = namedtuple(
        'PaysAffichage', ['Nom', 'Continent', 'ChefEtat'])
    table_affichage = [PaysAffichage(p[0], p[1], p[2]) for p in pays]
    return table_affichage


table_pays_continent_chef = construire_table_pays_continent_chef(pays)
afficher_table(table_pays_continent_chef, "Noms des pays, Continent, ChefEtat")


### Y a un monde fou ici!

Donner la table de tous les pays dont la densité de population dépasse 500 habitants par km².

La table ne montrera que le nom du pays, sa population, sa superficie et sa densité de population.

In [24]:
def table_densite(pays):
    """
    Construit une table de pays dont la densité de population est supérieure à 500.
    Arguments :
        pays : Une liste de namedtuple Pays contenant les informations sur chaque pays.
    Returns :
        Une liste de namedtuple PaysDensite contenant le nom, la population, la superficie et la densité de population de chaque pays.
    """
    PaysDensite = namedtuple(
        'PaysDensite', ['Nom', 'Population', 'Aire', 'Densite'])

    table_affichage = [PaysDensite(
        "Nom", "Population", "Aire", "Densite")]

    table_affichage += [PaysDensite(p.Nom, p.Population, p.Aire, p.Population / p.Aire)
                        for p in pays if p.Population / p.Aire > 500]

    return table_affichage



table_pays_densite = table_densite(pays)
afficher_table(table_pays_densite,
               "Noms des pays, Population, Aire, Densite de population")


### Plusieurs critères

Donner la table de tous les pays dont le PIB dépasse 100 000 et dont l'espérance de vie est inférieure à 78 ans.

In [25]:
pib_espvie = [(p.Nom, p.PIB, p.EsperanceVie) for p in pays if p.PIB > 100000 and p.EsperanceVie < 78]
afficher_table(pib_espvie, "Les pays avec un PIB supérieur à 100 000 et une espérance de vie inférieur à 78 ans")

Donner la table de toutes les républiques ayant accédé à l'indépendance après la 2ème guerre mondiale.

In [27]:
respublica_wwii = [(p.Nom, p.FormeGouvernement, p.AnneeIndependance) for p in pays if p.FormeGouvernement == "Republic" and p.AnneeIndependance > 1945]
afficher_table(respublica_wwii, "Les pays qui sont une république et qui sont indépendant après la seconde guerre mondiale")

## Un peu de calcul

### C'est grand comment l'Amérique ?

Donnez la surface totale de l'Amérique du nord.

In [9]:
aire_northusa = 0
for p in pays:
    if p.Region == "North America":
        aire_northusa += p.Aire
print(f"{aire_northusa}km²")

21500515.0km²


### Recensement mondial

Construire une table dont les lignes représentent les différents continents existants. Cette table aura deux colonnes: le nom du continent et sa population totale.

In [28]:
def construire_table_continents(pays):
    """
    Construit une table de continents avec leur population totale.
    Arguments :
        pays : Une liste de namedtuple Pays contenant les informations sur chaque pays.
    Returns :
        Une liste de namedtuple Continents contenant le nom du continent et sa population totale.
    """
    Continents = namedtuple('Continents', ['Region', 'Population'])

    continents_population = {p.Region: 0 for p in pays}

    for p in pays:
        continents_population[p.Region] += p.Population

    return [Continents(region, population) for region, population in continents_population.items()]

table_continents = construire_table_continents(pays)
afficher_table(table_continents, "Noms des continents, Population")


### La classe moyenne

Calculer la moyenne des PIB de tous les pays.

In [29]:
somme_pib = 0
nombre_pays = 0

for p in pays:
    somme_pib += p.PIB
    nombre_pays += 1

moyenne_pib = somme_pib // nombre_pays

print(f"La moyenne des PIB de tous les pays est : {moyenne_pib}$")


La moyenne des PIB de tous les pays est : 122823.0$


### Des croissants?

Trouvez le pays dont l'évolution du PIB est la plus faible en pourcentage (calcul: `(PIB - PIBAncien) / PIBAncien * 100`). Affichez le nom du pays et l'évolution de son PIB en pourcentage.

In [30]:
def pays_evolution_pib_faible(pays):
    """
    Calcule l'évolution du PIB la plus faible parmi une liste de pays
    """
    # Initialisation de la valeur minimale
    faible_evolution = None
    nom_pays_faible = None
    # On parcourt chaque pays de la table
    for p in pays:
        # On calcule l'évolution du PIB en pourcentage
        if p.PIBAncien is not None:
            evolution = (p.PIB - p.PIBAncien) / p.PIBAncien * 100
            # Si cette évolution est la plus faible rencontrée jusqu'à présent, on met à jour le résultat
            if faible_evolution is None or evolution < faible_evolution:
                faible_evolution = evolution
                nom_pays_faible = p.Nom
    # On retourne le résultat
    return faible_evolution, nom_pays_faible


# Calculer l'évolution du PIB la plus faible
pourcentage, nom_pays_faible = pays_evolution_pib_faible(pays)

pays = chargePays("pays.csv")

# Affichage du résultat
if pourcentage is not None and nom_pays_faible is not None:
    print("La plus faible évolution du PIB est de {}% pour le pays {}".format(
        pourcentage, nom_pays_faible))
else:
    print("Aucun pays avec un PIB ancien n'a été trouvé")


La plus faible évolution du PIB est de -60.47385605715295% pour le pays Indonesia


C'est la fin de ce TP, mais maintenant que vous savez rechercher de dans la table selon certains critères, vous pouvez imaginer vos propres questions et chercher à y répondre.